In [37]:
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from webdriver_manager.firefox import GeckoDriverManager
import re

#Create Driver

In [38]:
firefox_options = Options()
firefox_options.add_argument('--headless')
firefox_options.add_argument('--no-sandbox')
driver = webdriver.Firefox(options = firefox_options, executable_path=GeckoDriverManager().install())


In [39]:
page_url = 'https://witcher.fandom.com/wiki/Category:Characters_in_the_stories'
driver.get(page_url)

#Find Books

In [40]:
# Get all elements with the title attribute
books_elements = driver.find_elements_by_class_name('category-page__member-link')

In [41]:
books = []
for element in books_elements:
    book_url = element.get_attribute('href')
    book_name = element.text
    books.append({'book': book_name, 'url': book_url})

In [42]:
books

[{'book': 'Category:Baptism of Fire characters',
  'url': 'https://witcher.fandom.com/wiki/Category:Baptism_of_Fire_characters'},
 {'book': 'Category:Blood of Elves characters',
  'url': 'https://witcher.fandom.com/wiki/Category:Blood_of_Elves_characters'},
 {'book': "Godamba Thaess'en",
  'url': 'https://witcher.fandom.com/wiki/Godamba_Thaess%27en'},
 {'book': 'Category:Season of Storms characters',
  'url': 'https://witcher.fandom.com/wiki/Category:Season_of_Storms_characters'},
 {'book': 'Category:Something Ends, Something Begins characters',
  'url': 'https://witcher.fandom.com/wiki/Category:Something_Ends,_Something_Begins_characters'},
 {'book': 'Category:Sword of Destiny characters',
  'url': 'https://witcher.fandom.com/wiki/Category:Sword_of_Destiny_characters'},
 {'book': 'Category:Szpony i kły characters',
  'url': 'https://witcher.fandom.com/wiki/Category:Szpony_i_k%C5%82y_characters'},
 {'book': 'Category:Tales from the world of The Witcher characters',
  'url': 'https://wi

In [43]:
characters = []

for book in books:
    driver.get(book['url'])
    character_elements= driver.find_elements_by_class_name('category-page__member-link')
    for element in character_elements:
        characters.append({'book': book['book'], 'character': element.text})